In [5]:
import numpy as np
import pandas as pd

In [6]:
flow_index = {}
website_index = {}

In [7]:
df = pd.read_excel("flow.jsonN final.xlsx")

In [8]:
df.head()

,website,flow.json
0,Self-Improvement and Personal Development Blog...,NaN
1,History and Historical Archives:\n\nExamples: ...,NaN
2,Virtual Pet Games and Tamagotchi-style Apps:\n...,NaN
3,Educational Games and Learning Apps:\n\nExampl...,NaN
4,Freelance Writing and Content Creation Platfor...,NaN


In [9]:
cases = []

In [10]:
for index, row in df.iterrows():
  flow_index[index] = row["flow.json"]
  website_index[index] = row["website"]
  cases.append(row['website'])

In [11]:
# class for vector embedding and similarity search

import numpy as np

class VectorStore:
    def __init__(self):
        self.vector_data = {}
        self.vector_index = {}

    def add_vector(self, vector_id, vector):
        self.vector_data[vector_id] = vector
        self._update_index(vector_id, vector)

    def get_vector(self, vector_id):
        return self.vector_data.get(vector_id)

    def _update_index(self, vector_id, vector):
        # In this simple example, we use brute-force cosine similarity for indexing
        for existing_id, existing_vector in self.vector_data.items():
            similarity = np.dot(vector, existing_vector) / (np.linalg.norm(vector) * np.linalg.norm(existing_vector))
            if existing_id not in self.vector_index:
                self.vector_index[existing_id] = {}
            self.vector_index[existing_id][vector_id] = similarity

    def find_similar_vectors(self, query_vector, num_results=5):
        results = []
        for vector_id, vector in self.vector_data.items():
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            results.append((vector_id, similarity))

        # Sort by similarity in descending order
        results.sort(key=lambda x: x[1], reverse=True)

        # Return the top N results
        return results[:num_results]

In [12]:
vector_store = VectorStore()

In [13]:
# tokenization and vocabulary creation

vocabulary = set()
for curr_web in cases:
    tokens = curr_web.lower().split()
    vocabulary.update(tokens)

In [14]:
# Assign unique indices to words in the vocabulary
word_to_index = {word: i for i, word in enumerate(vocabulary)}

In [15]:
# Vectorization
case_vectors = {}
for curr_case in cases:
    tokens = curr_case.lower().split()
    vector = np.zeros(len(vocabulary))
    for token in tokens:
        vector[word_to_index[token]] += 1
    case_vectors[curr_case] = vector

In [16]:
# Storing in VectorStore
for curr_web, vector in case_vectors.items():
    vector_store.add_vector(curr_web, vector)

In [31]:
# Searching for Similarity
query_sentence = "weateher prediction application"
query_vector = np.zeros(len(vocabulary))
query_tokens = query_sentence.lower().split()
for token in query_tokens:
    if token in word_to_index:
        query_vector[word_to_index[token]] += 1

similar_sentences = vector_store.find_similar_vectors(query_vector, num_results=1)

In [32]:
similar_sentences

[('Secure Software Development and AppSec Platforms:\n- Example: OWASP (Open Web Application Security Project)\n- Purpose: Resources and tools for secure software development and application security.',
  0.3086066999241838)]

In [ ]:
#